In [1]:
!pip install transformers
!pip install datasets

In [2]:
import pandas as pd
import string
import regex as re

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

def tokenize(samples):
    return tokenizer(samples['content'], truncation=True, padding='max_length', return_tensors='pt')


tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [4]:
df = pd.read_csv("/kaggle/input/nlp-data/data.csv")
df

,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5
...,...,...,...
31455,Không đáng tiền,NEG,1
31456,Quần rất đẹp,POS,5
31457,Hàng đẹp đúng giá tiền,POS,5
31458,Chất vải khá ổn,POS,4


In [5]:
df = df.dropna()

In [6]:
import regex as re

emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def clean_text(text):
    text = text.lower()
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(r'([a-z]+?)\1+',r'\1', text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    # text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
    # text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
    text = text.strip()
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
    text = re.sub(r"\s+", " ", text)
    return text

def map_label(label):
    label_map = {
        "POS": 1,
        "NEG": 0,
        "NEU": 2
    }
    return label_map[label]
df.loc[:, "content"] = df["content"].map(lambda text: clean_text(text))
df.loc[:, "label"] = df["label"].map(lambda label: map_label(label))
df = df.dropna()

In [7]:
df=df.drop_duplicates("content")
df = df.drop(df[df["content"].map(len) < 2].index)
df = df.dropna()

In [8]:
df

,content,label,start
0,áo bao đẹp ạ,1,5
1,tuyệt vời,1,5
2,2day ao khong giong trong,0,1
3,"mùi thơm , bôi lên da mềm da",1,5
4,"vải đẹp , dày dặn",1,5
...,...,...,...
31452,"chất tốt , shop phục vụ rất tốt",1,5
31454,"hàng y hình , đóng gói ko cẩn thận nên quả ngự...",2,3
31457,hàng đẹp đúng giá tiền,1,5
31458,chất vải khá ổn,1,4


In [9]:
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25608 [00:00<?, ? examples/s]

In [10]:
tokenized_datasets

Dataset({
    features: ['content', 'label', 'start', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25608
})

In [11]:
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)  # 20% for test set

# Access the train and test sets
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3)

2024-08-10 16:57:34.432330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 16:57:34.432471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 16:57:34.548118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from sklearn.metrics import accuracy_score
import numpy as np

# Định nghĩa hàm tính toán metrics
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(labels, pred)
    return {"accuracy": accuracy}

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: WARNING URL not available in offline run
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.813800,0.686928,0.716517
2,0.653500,0.687048,0.738383
3,0.613900,0.601346,0.744631
4,0.549200,0.607851,0.756931


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=2564, training_loss=0.6390061981034539, metrics={'train_runtime': 2707.0693, 'train_samples_per_second': 30.27, 'train_steps_per_second': 0.947, 'total_flos': 1.0780282951151616e+16, 'train_loss': 0.6390061981034539, 'epoch': 4.0})

In [17]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6078505516052246,
 'eval_accuracy': 0.7569308863725107,
 'eval_runtime': 53.4026,
 'eval_samples_per_second': 95.913,
 'eval_steps_per_second': 3.015,
 'epoch': 4.0}